# 【事前準備】安裝Python以及VScode
## Python
Python 建議版本 : 3.9 ( 有時候太新會有一些問題喔! )

* *★安裝過程注意★*
* **在一開始安裝時，務必把下面的□Add python3.9 to PATH打勾** 
> <img src="./pic/python_install_pic1.png" width="40%">

* **在選擇" Option Features "時，務必把□pip打勾** 
> <img src="./pic/python_install_pic2.png" width="40%">

## VScode 擴充套件
> <img src="./pic/extension_install_pic.png" width="40%">

(Python的都建議安裝一下)
* Pylance、Python、Python Environment Manager
* Python Extension Pack、Python for VSCode

(其他推薦)
* Intellicode、Intellicode API Usage Example
* Jupyter、Jupyter Notebook Renderers (可以看老師的程式碼)
*** 

# 【事前準備】建立資料夾
* Step1 : 在"桌面"建立新資料夾，取名"gesture_control_project"
* Step2 : 開啟VScode，將"gesture_control_project"這個資料夾拉到工作區
* Step3 : 在工作區右鍵，"新增檔案"，取名"gesture_volume_control.py"
*** 

# 【事前準備】安裝套件!
## 請在終端機輸入以下指令
> pip install opencv-python      #圖像處理、電腦視覺以及圖型識別程式庫

> pip install mediapipe          #手勢辨識套件

> pip install numpy              #矩陣計算套件

> pip install pycaw              #音量控制套件
* **安裝完套件後，建議重新開一次編輯器!**

*** 
#### ⚠️不知道終端機在哪?
* 方法一 : VScode工具列 >> 終端機(T) >> 新增終端
* 快捷鍵 : Ctrl+Shift+`
--- 


# 【PART1 打開電腦的靈魂之窗-讀取鏡頭】
## 引入函式庫
* 點開"gesture_volume_control.py"，開始編輯
* 首先，我們要將我們可能會用到的套件都引入進來。

In [ ]:
import cv2  #opencv 影像處理的套件
import numpy as np #矩陣運算
import time #用來取得電腦時間
import mediapipe as mp #偵測模型
import math #偵測模型

#實現音量控制所需的套件:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume    #音量控制套件

## (1)讀取鏡頭，並顯示出來
引入我們需要使用的工具後，就開始實作吧!

* 首先，我們要先取得攝影機的影像。並且可以用cap.set()調整長寬。
>       cam_width , cam_height = 640, 480
        cap = cv2.VideoCapture(0)
        cap.set(3, cam_width)   #調整影像寬度
        cap.set(4, cam_height)   #調整影像長度

* 接下來，建立一個不斷執行的while迴圈。
* cap.read()這個函式可以回傳**ret, img**兩個值，**ret**:有沒有回傳資料(0:沒有，1:有)，以及**img**:影像資料。
* 如果ret存在，那麼就可以用**cv2.imshow()**這個函式開啟一個叫做'img'的視窗，將img這個影像顯示出來!
* **ord('q')**的意思等同於"在鍵盤上按q(注意是小寫喔!)"。所以，**if cv2.waitKey(10) & 0xFF == ord('q')**意思就是，如果我按了q，就終止while迴圈，讓程式中止!
>   while True:
        ret, img = cap.read()
        if ret:
            cv2.imshow('img', img) #顯示畫面
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

#### 📁為什麼要& 0xFF:
問得好!

cv2.waitkey是OpenCV內置的函式，用途是在給定的時間內(單位毫秒)等待使用者的按鍵觸發，否則持續循環。
ord(' ')可以將字符轉化為對應的整數(ASCII碼)。

0xFF是十六進制常數，二進制值為11111111。這個寫法只留下原始的最後8位，和後面的ASCII碼對照。


總之此處是為了防止BUG~

In [ ]:
##攝影機設定##
cam_width , cam_height = 640, 480
cap = cv2.VideoCapture(0) #填入0或1試試看
cap.set(3, cam_width)   #調整影像寬度
cap.set(4, cam_height)   #調整影像長度

##迴圈區域##
while True:
    ret, img = cap.read()
    
    if ret:
        cv2.imshow('img', img) #顯示畫面

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

### 執行結果: 
> <img src="./pic./read_success.png" width="40%">

# 【PART2 實現即時手部偵測】

## (1)mediapipe手部偵測模型設定
* 讀取到鏡頭後，就可以進入主題囉!! ><
* **在迴圈前面**，我們從mediapipe中引用他們的類別(Class)，存到一個叫做**mphands**的變數中
* 從這個模型中，我們設定好手部辨識模型的參數，存到一個叫做**hands**的變數中。(hands的參數有很多，之後我們再來玩玩看。)
* 我們也引用mediapipe中的繪畫工具，存到一個叫做**mpDraw**的變數中。

In [ ]:
##手部模型偵測參數設定、功能引入##
mphands = mp.solutions.hands  #使用mediapipe裡的手部辨識功能
hands = mphands.Hands(min_detection_confidence=0.5,min_tracking_confidence=0.5) #設定手部辨識模型
mpDraw = mp.solutions.drawing_utils #繪畫工具

## (2)於迴圈中使用手部偵測模型

* 手部偵測需要偵測的是RGB的圖片，但因為opencv讀到的圖片都式BGR型式的圖片。
* 所以要先用**img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)**讓BGR的圖片，轉換成RGB的形式。
* 接下來把處理好的圖片(img_rgb)，利用**hands.process()**放到手部偵測模型當中，並把得到的結果用result存起來。


* result裡面存了很多資料，其中我們可以用.multi_hand_landmarks，去讀取手部21個座標(x,y,z)的串列(list)。
* print出來會看到一堆字。如果印出**None**表示，沒有偵測到手。

#### 📁為什麼要用BGR，RGB不是比較常見嗎??:
以前的年代比較常使用BGR，因此opencv這個套件也使用了BGR模式。雖然現在RGB模式比較流行了，但opencv開發已久，一時間要改回來很~~~麻煩。

所以正所謂:「前人種樹，後人乘涼」，如果沒特別原因，應該不需要砍了那棵樹打掉重練吧ww

https://www.zhihu.com/question/264044792

In [ ]:
##迴圈區域##
while True:
    ret, img = cap.read()       #讀取鏡頭
    
    if ret:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      #bgr轉rgb
        result = hands.process(img_rgb)
        print(result.multi_hand_landmarks)
        
        cv2.imshow('img', img) #顯示畫面

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

## (3)畫出座標點
* 有了座標之後，就可以在視窗的影像上把點點畫出來。
* 自己寫程式畫出來可能有點麻煩，剛好mediapipe也有提供畫點的函式! (讚🎉🎉
* 我們剛剛已經把這個工具存在**mpdraw**了，我們直接使用他吧!


* 我們先寫一個迴圈，如果有偵測到手，用for迴圈把每一隻手(用**handLams**當index)的點點畫出來。

In [ ]:
##迴圈區域##
while True:
    ret, img = cap.read()       #讀取鏡頭
    
    if ret:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      #bgr轉rgb
        result = hands.process(img_rgb)
        #print(result.multi_hand_landmarks)

        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:     #畫出每個手的座標及連線、設定樣式
                mpDraw.draw_landmarks(img, handLms)
        
        cv2.imshow('img', img) #顯示畫面

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
## [執行看看] ##

### 執行結果: 
> <img src="./pic/red_spot.png" width="40%">

## (4)畫出骨架(連接座標點)
* 只有點點看起來實在很像十八銅人的穴位圖... 所以，我們趕快把骨架連起來吧!
* 這時在**mpDraw.draw_landmarks()**函式中，加入第三個參數**mphands.HAND_CONNECTIONS**。

In [ ]:
##迴圈區域##
while True:
    ret, img = cap.read()       #讀取鏡頭
    
    if ret:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      #bgr轉rgb
        result = hands.process(img_rgb)
        #print(result.multi_hand_landmarks)

        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:     #畫出每個手的座標及連線、設定樣式
                mpDraw.draw_landmarks(img, handLms,mphands.HAND_CONNECTIONS)
        
        cv2.imshow('img', img) #顯示畫面

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

### 執行結果: 
> <img src="./pic/connection.png" width="40%">

# 【PART3 樣式調整、加上標號】
## (1)樣式設定
* 如果大家有點設計涵養，覺得點點跟線的樣式太醜，可以加入第四個參數(點的樣式)，第五個參數(線的樣式)
* 不過，這裡設定樣式的方式比較特別，我們要先寫一個**樣式設定**在while迴圈前面。
* 利用**mpDraw.DrawingSpec()**我們可以設定座標點及連接線的顏色、粗細。


* 然後，我再將**handLms_style**、**handCon_style**放到**mpDraw.draw_landmarks()**函式中
>   mpDraw.draw_landmarks(img, handLms,mphands.HAND_CONNECTIONS, handLms_style, handCon_style)

⚠️注意，這邊的顏色設定是BGR模式喔~

In [ ]:
##手部骨架樣式設定##
handLms_style = mpDraw.DrawingSpec(color=(73,93,70), thickness=6)  #手座標樣式
handCon_style = mpDraw.DrawingSpec(color=(255,250,240), thickness=3) #手連接樣式

In [ ]:
##迴圈區域##
while True:
    ret, img = cap.read()       #讀取鏡頭
    
    if ret:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      #bgr轉rgb
        result = hands.process(img_rgb)
        #print(result.multi_hand_landmarks)

        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:     #畫出每個手的座標及連線、設定樣式
                mpDraw.draw_landmarks(img, handLms,mphands.HAND_CONNECTIONS, handLms_style, handCon_style)
        
        cv2.imshow('img', img) #顯示畫面
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

### 執行結果: 
> <img src="./pic/color.png" width="40%">

## (2)加上標號
* 現在我們都把手的座標點跟連接線畫出來了。
* 但我們要知道座標，才可以做各種應用嘛。所以接下來，就是要把21個點的座標找出來!!


* **handLms.landmark**當中，存了21個點的x座標及y座標。
* 我們用一個for迴圈，搭配列舉enumerate()函式，把每一隻手的21個座標print出來。

>       for i, lm in enumerate(handLms.landmark):       #輸出每個手的座標點 
            print(i, lm.x, lm.y)

#### 📁列舉函式enumerate():
函式enumerate()是以index為所以，將串列、元組或字串中的數據列出來，一般用於for迴圈。
https://www.runoob.com/python/python-func-enumerate.html

In [ ]:
##迴圈區域##
while True:
    ret, img = cap.read()       #讀取鏡頭
    
    if ret:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      #bgr轉rgb
        result = hands.process(img_rgb)
        #print(result.multi_hand_landmarks)

        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:     #畫出每個手的座標及連線、設定樣式
                mpDraw.draw_landmarks(img, handLms,mphands.HAND_CONNECTIONS, handLms_style, handCon_style)
                
                for i, lm in enumerate(handLms.landmark):       #輸出每個手的座標點 
                    print(i, lm.x, lm.y)
        
        cv2.imshow('img', img) #顯示畫面
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

* 我們確認一下，可以發現我們印出了0到20的座標點。但這個座標有一點怪怪的，數值都在0到1之間。
* 其實，這個數值指是**比例**。假設我們畫面是100*100，那0.6就大概在寬度60的地方
* 所以要得到真正的座標，只要**乘上視窗的高度跟寬度**即可。


* 在img裡面**img.shape[0]存了影像的寬，img.shape[1]存了影像的高**。
* 我們先在**「if result.multi_hand_landmarks:」**這個判斷迴圈外，設定這兩個變數:

>       img_height = img.shape[0]  #得到影像的高
        img_width = img.shape[1]   #得到影像的寬


* 然後，回去for...enumerte()迴圈，將x,y比例座標分別乘上寬度與長度，並且用int()取整函數得到整數。得到的**xPos,yPos**也就是真正影像上的座標位置。
>       xPos = int(lm.x * img_width)    #比例乘上寬度
        yPos = int(lm.y * img_height)   #比例乘上高度
        print(i, xPos, yPos)

In [ ]:
##迴圈區域##
while True:
    ret, img = cap.read()       #讀取鏡頭
    
    if ret:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      #bgr轉rgb
        result = hands.process(img_rgb)
        #print(result.multi_hand_landmarks)
        
        img_height = img.shape[0]  #得到影像的高
        img_width = img.shape[1]   #得到影像的寬

        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:     #畫出每個手的座標及連線、設定樣式
                mpDraw.draw_landmarks(img, handLms,mphands.HAND_CONNECTIONS, handLms_style, handCon_style)
                
                for i, lm in enumerate(handLms.landmark):       #輸出每個手的座標點 
                    xPos = int(lm.x * img_width)    #比例乘上寬度
                    yPos = int(lm.y * img_height)   #比例乘上高度
                    print(i, xPos, yPos)
        
        cv2.imshow('img', img) #顯示畫面
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

* 有了座標後，我們用就可以用cv2.putText()這個函式，把座標數字標上去!
* **cv2.putText(放在哪張圖, 要放的文字, 座標, 字形, 顏色, 粗細)**

>   cv2.putText(img, str(i), (xPos-25, yPos+5), cv2.FONT_HERSHEY_COMPLEX, 0.4, (0,69,255), 1)   #加上標籤

In [ ]:
##迴圈區域##
while True:
    ret, img = cap.read()       #讀取鏡頭
    
    if ret:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      #bgr轉rgb
        result = hands.process(img_rgb)
        #print(result.multi_hand_landmarks)
        
        img_height = img.shape[0]  #得到影像的高
        img_width = img.shape[1]   #得到影像的寬

        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:     #畫出每個手的座標及連線、設定樣式
                mpDraw.draw_landmarks(img, handLms,mphands.HAND_CONNECTIONS, handLms_style, handCon_style)
                
                for i, lm in enumerate(handLms.landmark):       #輸出每個手的座標點 
                    xPos = int(lm.x * img_width)    #比例乘上寬度
                    yPos = int(lm.y * img_height)   #比例乘上高度
                    cv2.putText(img, str(i), (xPos-25, yPos+5), cv2.FONT_HERSHEY_COMPLEX, 0.4, (0,69,255), 1)   #加上標籤
        
        cv2.imshow('img', img) #顯示畫面
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

## (3)把大拇指的點點畫上去。
* 觀察一下，第4、8、12、16、20個點，分別是哪個部位?
* Ans: 一定是大拇指的啦!👍️👍️👍️👍️


* 如果我們要特別標註這些點，可以用cv2.circle()直接在第四點畫一個圈。
* **cv2.circle(畫在哪裡, 座標, 半徑, 顏色, 粗度可以直接填滿)**
* 現在把所有手指頭都填滿吧!! (補充: | 這個標號是邏輯中的"或"的意思)

>    if (i == 4)|(i == 8)|(i == 12)|(i == 16)|(i == 20):  
        cv2.circle(img, (xPos, yPos), 9, (34,139,34), cv2.FILLED)

In [ ]:
##迴圈區域##
while True:
    ret, img = cap.read()       #讀取鏡頭
    
    if ret:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      #bgr轉rgb
        result = hands.process(img_rgb)
        #print(result.multi_hand_landmarks)
        
        img_height = img.shape[0]  #得到影像的高
        img_width = img.shape[1]   #得到影像的寬

        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:     #畫出每個手的座標及連線、設定樣式
                mpDraw.draw_landmarks(img, handLms,mphands.HAND_CONNECTIONS, handLms_style, handCon_style)
                
                for i, lm in enumerate(handLms.landmark):       #輸出每個手的座標點 
                    xPos = int(lm.x * img_width)    #比例乘上寬度
                    yPos = int(lm.y * img_height)   #比例乘上高度
                    cv2.putText(img, str(i), (xPos-25, yPos+5), cv2.FONT_HERSHEY_COMPLEX, 0.4, (0,69,255), 1)   #加上標籤
                    
                    # 畫拇指!
                    if (i == 4)|(i == 8)|(i == 12)|(i == 16)|(i == 20):  
                        cv2.circle(img, (xPos, yPos), 9, (34,139,34), cv2.FILLED)
        
        cv2.imshow('img', img) #顯示畫面
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

### 執行結果: 
> <img src="./pic/complete.png" width="40%">

### 完成後，幫自己比一個讚吧!

# 【PART4 DJ Drop The Beat-用手勢控制音量大小!】
* 如果把上面的實作出來，就可以實現很多功能喔!
* 這邊教你做一個可以用手勢調整音樂音量專案，沒當過DJ沒關係，這次體驗一次XD!

#### P.S. 其實還有很多功用啦~ 像是智能健身教練、手語辨識、簡易音遊，只要妥當設計，這個真的超好玩der

## (1)設計專案需求
* 在做任何專案以前，先思考自己有哪些**"功能需求"**，並且思考**"要如何以程式實現"**。


* 以下是這次專案會想要的東西:
1. **顯示幀率(frame rate):** 為了判斷我的畫面有沒有很卡，所以想要放上去
2. **音量指示條:** 有個指示條感覺比較直觀、清楚，看起來也更炫(?)
3. **音量百分比:** 可以確切看到音量大小的數值。
4. **拇指跟食指連線:** 因為想要用"拇指跟食指的距離"，來調整音樂的大小聲。
5. **中點:** 想要在"最大聲"及"最小聲"的時候，讓中點變顏色來給予提醒。


* 除了設計**邏輯功能**之外，也要思考如何將這些東西**可視化**喔!

> <img src="./pic/design.png" width="80%">

* **在進到下個part之前，可以先跟同學討論看看可以怎麼修改原本的程式碼，實現這個專案!**

# 【PART5 各項參數的初始化設定】
* 在while迴圈前，我們要加入一些初始化設定。

## (1) 幀率的參數設定
* 為了要計算每秒有多少幀，我需要記錄現在的時間點(current_time)，還有上個時間點(previous_time)。
* 至於如何計算，會在後面解釋~
>       current_time = 0
        previous_time = 0   

## (2)音量控制相關設定
* 用**vol、volbar、volper**分別記錄**音量大小、音量指示條、音量百分比**的數值。
>       vol = 0 #初始化音量大小
        volbar = 400 #初始化音量指示條頂的位置
        volper = 0  #初始化音量百分比數值

## (3)取得裝置的音量範圍
* 取得裝置的音源。這部分暫時不用理解，直接複製貼上即可~
>       devices = AudioUtilities.GetSpeakers()
        interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
        volume = cast(interface, POINTER(IAudioEndpointVolume))


* 利用**volume.GetVolumeRange()**可以取得裝置音量範圍資料，其中音量最小值存在volRange[0]，最大值存在volRange[1]
* 我們把它print出來，記錄在註解裡。
>       volRange = volume.GetVolumeRange()  #取得裝置音量範圍資料
        minVol, maxVol = volRange[0], volRange[1]   #取得裝置音量最大值、最小值
        print(volume.GetVolumeRange())  #輸出音量範圍   (-96,0,後面這項不用理他)

In [ ]:
##FPS計算參數設定##
current_time = 0
previous_time = 0

##音量控制相關設定##
vol = 0 #初始化音量大小
volbar = 400 #初始化音量指示條頂的位置
volper = 0  #初始化音量百分比數值

##取得裝置的音量範圍##
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))

volRange = volume.GetVolumeRange()  #取得裝置音量範圍資料
minVol, maxVol = volRange[0], volRange[1]   #取得裝置音量最大值、最小值
print(volume.GetVolumeRange())  #輸出音量範圍   (-96,0,後面這個不用理他)

# 【PART6 控制功能設計】
* 將參數初始化之後，就可以開始設計功能的程式囉!

## (1)計算拇指、食指兩點的長度，並設定觸發事件
* 跟著下圖的步驟，我們一步步修改原本的程式碼。
> <img src="./pic/event.png" width="80%">


* 在記錄座標點的迴圈內，我們需要取得**"第四點"跟"第八點"的座標**。

>         if (i == 4):
            x4, y4 = xPos, yPos
        if (i == 8):
            x8, y8 = xPos, yPos


* 在記錄座標點的迴圈外面，計算**中點座標**。然後**計算拇指、食指兩點的長度**。
>       xm , ym = (x4+x8)//2 , (y4+y8)//2   #取得中點座標
        length = math.hypot(x8-x4, y8-y4)   #用square root計算長度
        

* 在畫手的迴圈外面，設定觸發事件。
        if (length <= 25)|(length >= 185):        #手指捏起來的時候，中點變色
            cv2.circle(img, (xm, ym), 9, (34,34,78), cv2.FILLED)

In [ ]:
##迴圈區域##
while True:
    ret, img = cap.read()
    #hand_detection
    if ret:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      #bgr轉rgb
        result = hands.process(img_rgb)

        img_height = img.shape[0]
        img_width = img.shape[1]

        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:     #畫出每個手的座標及連線、設定樣式
                mpDraw.draw_landmarks(img, handLms,mphands.HAND_CONNECTIONS, handLms_style, handCon_style)
                    
                for i, lm in enumerate(handLms.landmark):       #輸出每個手的座標點 
                    xPos = int(lm.x * img_width)    #比例乘上寬度
                    yPos = int(lm.y * img_height)   #比例乘上高度
                    
                    #記錄拇指與食指座標
                    if (i == 4):
                        x4, y4 = xPos, yPos
                    if (i == 8):
                        x8, y8 = xPos, yPos
                
                xm , ym = (x4+x8)//2 , (y4+y8)//2   #取得中點座標
                
                #計算拇指、食指兩點的長度，並設定觸發事件
                length = math.hypot(x8-x4, y8-y4)   #用square root計算長度

            if (length <= 25)|(length >= 200):        #手指捏起來的時候，中點變色 (設定觸發事件)
                cv2.circle(img, (xm, ym), 9, (34,34,78), cv2.FILLED)
                
    cv2.imshow("img",img)

    if cv2.waitKey(1) == ord('q'):
        break

## (2)映射調整
* 接下來想想看，如何用"手張多大"去影響"音量多大聲"、"指示條多長"、"百分比數字多大"呢?
* 像這種從**"某個變數"去對應到"另一個變數"**的關係我們在數學上稱為**「映射」**。跟函數關係很類似!

> <img src="./pic/mapping.png" width="80%">

* 我們這次用的方法叫做**「線性插值法」**，有興趣的同學下面資料可以點進去看。
* np.interp()這個函式可以在給予"拇指與食指長度"後，自動輸出一個在"特定範圍內"的數值。
* 舉例: 長度為105時，音量百分比對應到的是45%。

>         vol = np.interp(length,[25,200],[-30,maxVol])   #音量映射調整
        volbar = np.interp(length,[25,200],[400,150])   #指示條長度映射調整
        volper = np.interp(length,[25,200],[0,100])   #指示條數值映射調整


* 而音量設定的部分，就用volume.SetMasterVolumeLevel(vol, None)來實現!

>   volume.SetMasterVolumeLevel(vol, None)


#### 📁線性插值:
https://zh.wikipedia.org/zh-tw/%E7%BA%BF%E6%80%A7%E6%8F%92%E5%80%BC
#### 📁numpy.interp()用法:
https://blog.csdn.net/hfutdog/article/details/87386901

In [ ]:
#映射調整
            vol = np.interp(length,[25,200],[-30,maxVol])   #音量映射調整
            volbar = np.interp(length,[25,200],[400,150])   #指示條長度映射調整
            volper = np.interp(length,[25,200],[0,100])   #指示條數值映射調整
            volume.SetMasterVolumeLevel(vol, None)  #設定音量

## (3)計算幀率
* 我們可能沒辦法直接計算一秒中有多少幀(frame)，但一秒幾次不就是**「頻率」**的概念嗎?!
* 那只要計算兩個畫面產生的週期是多長，再用「頻率為週期的倒數」，$F=\frac{1}{T}$，就可以算出幀率囉!

* 記得寫在**if ret:**這個判斷迴圈外。

> <img src="./pic/fps.png" width="80%">

In [ ]:
#計算幀率
    current_time = time.time()  #取得當下的時間
    fps = 1/(current_time-previous_time)    #算出"週期"，倒數後變成"頻率"
    previous_time = current_time    #記錄當下的時間，當作下次算週期的參考。


# 【PART7 功能可視化】
* 把功能邏輯都做完之後，還是要讓功能以圖形、文字顯示出來，以便於讓使用者更方便理解與使用。這種概念稱為**「可視化(Visualization)」**。

## (1)畫出拇指、食指、中點三個點，並連線。
* 我們把拇指、食指、中點三個點畫上去，並把線連起來吧~
* 記得**這段要加在觸發事件之前**喔!要不然觸發事件會被覆蓋掉~ 

In [ ]:
            #畫出拇指、食指、中指三個點，並連線
            cv2.circle(img, (x4, y4), 9, (105,165,218), cv2.FILLED)
            cv2.circle(img, (x8, y8), 9, (105,165,218), cv2.FILLED)
            cv2.circle(img, (xm, ym), 9, (105,165,218), cv2.FILLED)
            cv2.line(img,(x4, y4),(x8, y8),(105,165,218),3)

## (2)畫出指示框、指示條、顯示幀率、音量百分比
* 最後，在**if ret:**這個判斷迴圈外，把指示框、指示條、幀率、音量百分比也加上去
* 值得注意的是，cv2.putText()函式中，填入字串參數中有變數存在，因此前面要加f""，而變數的部分以大括弧{}框住。這個叫做**「字串格式化」**

#### 📁格式化的字串文本 (Formatted String Literals)
https://docs.python.org/zh-tw/3/tutorial/inputoutput.html

In [ ]:
#畫出指示框、指示條、顯示幀率、音量百分比
    cv2.rectangle(img,(50,150),(85,400),(105,165,218),3)  #指示條外框
    cv2.rectangle(img,(50,int(volbar)),(85,400),(105,165,218),cv2.FILLED)   #指示條填滿
    cv2.putText(img,f"FPS:{int(fps)}",(30,50), cv2.FONT_HERSHEY_DUPLEX, 1.5, (49,68,52), 3)  #fps標籤
    cv2.putText(img,f"{int(volper)}%",(40,450), cv2.FONT_HERSHEY_DUPLEX, 1,(105,165,218), 2)  #音量數值標籤

# 搭啦!大功告成囉!!!!!!!!~~🎊🎊🎉🎉🎉🥳🥳
### 快點一首音樂來玩玩看吧!

> <img src="./pic/final.png" width="80%">

In [ ]:
##迴圈區域##
while True:
    ret, img = cap.read()
    #hand_detection
    if ret:
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)      #bgr轉rgb
        result = hands.process(img_rgb)

        img_height = img.shape[0]
        img_width = img.shape[1]

        if result.multi_hand_landmarks:
            for handLms in result.multi_hand_landmarks:     #畫出每個手的座標及連線、設定樣式
                mpDraw.draw_landmarks(img, handLms,mphands.HAND_CONNECTIONS, handLms_style, handCon_style)
                    
                for i, lm in enumerate(handLms.landmark):       #輸出每個手的座標點 
                    xPos = int(lm.x * img_width)    #比例乘上寬度
                    yPos = int(lm.y * img_height)   #比例乘上高度
                    
                    #記錄拇指與食指座標
                    if (i == 4):
                        x4, y4 = xPos, yPos
                    if (i == 8):
                        x8, y8 = xPos, yPos
                
                xm , ym = (x4+x8)//2 , (y4+y8)//2   #取得中點座標
                
                #計算拇指、食指兩點的長度，並設定觸發事件
                length = math.hypot(x8-x4, y8-y4)   #用square root計算長度
                
            #映射調整
            vol = np.interp(length,[25,200],[-30,maxVol])   #音量映射調整
            volbar = np.interp(length,[25,200],[400,150])   #指示條長度映射調整
            volper = np.interp(length,[25,200],[0,100])   #指示條數值映射調整
            volume.SetMasterVolumeLevel(vol, None)  #設定音量    
                
            #畫出拇指、食指、中指三個點，並連線
            cv2.circle(img, (x4, y4), 9, (105,165,218), cv2.FILLED)
            cv2.circle(img, (x8, y8), 9, (105,165,218), cv2.FILLED)
            cv2.circle(img, (xm, ym), 9, (105,165,218), cv2.FILLED)
            cv2.line(img,(x4, y4),(x8, y8),(105,165,218),3)
            
            if (length <= 25)|(length >= 200):        #手指捏起來的時候，中點變色 (設定觸發事件)
                cv2.circle(img, (xm, ym), 9, (34,34,78), cv2.FILLED)
    
    #計算幀率
    current_time = time.time()  #取得當下的時間
    fps = 1/(current_time-previous_time)    #算出"週期"，倒數後變成"頻率"
    previous_time = current_time    #記錄當下的時間，當作下次算週期的參考。
    
    #畫出指示框、指示條、顯示幀率、音量百分比
    cv2.rectangle(img,(50,150),(85,400),(105,165,218),3)  #指示條外框
    cv2.rectangle(img,(50,int(volbar)),(85,400),(105,165,218),cv2.FILLED)   #指示條填滿
    cv2.putText(img,f"FPS:{int(fps)}",(30,50), cv2.FONT_HERSHEY_DUPLEX, 1.5, (49,68,52), 3)  #fps標籤
    cv2.putText(img,f"{int(volper)}%",(40,450), cv2.FONT_HERSHEY_DUPLEX, 1,(105,165,218), 2)  #音量數值標籤    
                
    cv2.imshow("img",img)

    if cv2.waitKey(1) == ord('q'):
        break